In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import tomllib
from pathlib import Path
from pandas import DataFrame
from common.utils import load_dataset, optimize_memory, get_params, DatasetType
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import joblib
import os
from xgboost import XGBRegressor

In [2]:
color = sns.color_palette()

# Google Brain - Ventilator Pressure Prediction

<br />
 
##### Overview

---
<div><p>What do doctors do when a patient has trouble breathing? They use a ventilator to pump oxygen into a sedated patient's lungs via a tube in the windpipe. But mechanical ventilation is a clinician-intensive procedure, a limitation that was prominently on display during the early days of the COVID-19 pandemic. At the same time, developing new methods for controlling mechanical ventilators is prohibitively expensive, even before reaching clinical trials. High-quality simulators could reduce this barrier. </p>
<p>Current simulators are trained as an ensemble, where each model simulates a single lung setting. However, lungs and their attributes form a continuous space, so a parametric approach must be explored that would consider the differences in patient lungs. </p>
<p>Partnering with Princeton University, the team at Google Brain aims to grow the community around machine learning for mechanical ventilation control. They believe that neural networks and deep learning can better generalize across lungs with varying characteristics than the current industry standard of PID controllers.  </p>
<p>In this competition, you’ll simulate a ventilator connected to a sedated patient's lung. The best submissions will take lung attributes compliance and resistance into account.</p>
<p>If successful, you'll help overcome the cost barrier of developing new methods for controlling mechanical ventilators. This will pave the way for algorithms that adapt to patients and reduce the burden on clinicians during these novel times and beyond. As a result, ventilator treatments may become more widely available to help patients breathe.</p>

##### Evaluation

---

The competition will be scored as the mean absolute error between the predicted and actual pressures during the inspiratory phase of each breath. The expiratory phase is not scored. The score is given by:

|X−Y|

where X
is the vector of predicted pressure and Y

is the vector of actual pressures across all breaths in the test set.
Submission File

For each id in the test set, you must predict a value for the pressure variable. The file should contain a header and have the following format:

<br/>

##### Submission File

----

For each id in the test set, you must predict a value for the pressure variable. The file should contain a header and have the following format:

```csv
id,pressure
1,20
2,23
3,24
etc.
```

##### Dataset Description

---

The ventilator data used in this competition was produced using a modified open-source ventilator connected to an artificial bellows test lung via a respiratory circuit. The diagram below illustrates the setup, with the two control inputs highlighted in green and the state variable (airway pressure) to predict in blue. The first control input is a continuous variable from 0 to 100 representing the percentage the inspiratory solenoid valve is open to let air into the lung (i.e., 0 is completely closed and no air is let in and 100 is completely open). The second control input is a binary variable representing whether the exploratory valve is open (1) or closed (0) to let air out.

In this competition, participants are given numerous time series of breaths and will learn to predict the airway pressure in the respiratory circuit during the breath, given the time series of control inputs.

![image](../docs/ventilator-diagram.png)

Each time series represents an approximately 3-second breath. The files are organized such that each row is a time step in a breath and gives the two control signals, the resulting airway pressure, and relevant attributes of the lung, described below.

<br>

<table>
  <thead>
    <tr>
      <th>File</th>
      <th>Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>train.csv</td>
      <td>The training set</td>
    </tr>
    <tr>
      <td>test.csv</td>
      <td>The test set</td>
    </tr>
    <tr>
      <td>sample_submission.csv</td>
      <td>A sample submission file in the correct format</td>
    </tr>
  </tbody>
</table>

<br>

<table>
  <thead>
    <tr>
      <th>Column</th>
      <th>Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>id</td>
      <td>Globally-unique time step identifier across an entire file</td>
    </tr>
    <tr>
      <td>breath_id</td>
      <td>Globally-unique time step for breaths</td>
    </tr>
    <tr>
      <td>R</td>
      <td>Lung attribute indicating how restricted the airway is (in cmH2O/L/S). This is the change in pressure per change in flow.</td>
    </tr>
    <tr>
      <td>C</td>
      <td>Lung attribute indicating how compliant the lung is (in mL/cmH2O). This is the change in volume per change in pressure.</td>
    </tr>
    <tr>
      <td>time_step</td>
      <td>The actual time stamp</td>
    </tr>
    <tr>
      <td>u_in</td>
      <td>The control input for the inspiratory solenoid valve. Ranges from 0 to 100.</td>
    </tr>
    <tr>
      <td>u_out</td>
      <td>The control input for the exploratory solenoid valve. Either 0 or 1.</td>
    </tr>
    <tr>
      <td>pressure</td>
      <td>The airway pressure measured in the respiratory circuit, measured in cmH2O.</td>
    </tr>
  </tbody>
</table>

In [3]:
train_df: DataFrame = load_dataset("ventilator-pressure-prediction", DatasetType.TRAIN, index=False)
train_df.head()

,breath_id,R,C,time_step,u_in,u_out,pressure
id,,,,,,,
1,1,20,50,0.000000,0.083334,0,5.837492
2,1,20,50,0.033652,18.383041,0,5.907794
3,1,20,50,0.067514,22.509278,0,7.876254
4,1,20,50,0.101542,22.808822,0,11.742872
5,1,20,50,0.135756,25.355850,0,12.234987


In [4]:
train_df.dtypes

breath_id      int64
R              int64
C              int64
time_step    float64
u_in         float64
u_out          int64
pressure     float64
dtype: object

In [5]:
train_df, na_list = optimize_memory(train_df)
train_df.dtypes

Memory usage of properties dataframe is : 368.408203125  MB
******************************
Column:  breath_id
dtype before:  int64
dtype after:  uint32
******************************
******************************
Column:  R
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  C
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  time_step
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  u_in
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  u_out
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  pressure
dtype before:  float64
dtype after:  float32
******************************
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  155.42221069335938  MB
This is  42.1875 % of

breath_id     uint32
R              uint8
C              uint8
time_step    float32
u_in         float32
u_out          uint8
pressure     float32
dtype: object

In [6]:
params: dict = {
    'learning_rate': 0.3,
    'booster': 'gbtree',
    'gamma': None,
    'min_child_weight': 3,
    'lambda': 0.5,
    'verbosity': 3,
    'tree_method': 'exact',
    'objective': 'reg:squarederror',
    'n_estimators': 100
}

model = XGBRegressor(**params)
get_params(model)

,Parameter,Value
0,objective,reg:squarederror
1,base_score,None
2,booster,gbtree
3,callbacks,None
4,colsample_bylevel,None
5,colsample_bynode,None
6,colsample_bytree,None
7,device,None
8,early_stopping_rounds,None
9,enable_categorical,False


In [7]:
test_df: DataFrame = load_dataset("ventilator-pressure-prediction", DatasetType.TEST, index=False)
test_df.head()

,breath_id,R,C,time_step,u_in,u_out
id,,,,,,
1,0,5,20,0.000000,0.000000,0
2,0,5,20,0.031904,7.515046,0
3,0,5,20,0.063827,14.651675,0
4,0,5,20,0.095751,21.230610,0
5,0,5,20,0.127644,26.320956,0
